# Handling Ambiguity

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

#Identify Ambiguous prompt

In [3]:
ambiguous_prompts = [
    "Tell me about the bank.",
    "What's the best way to get to school?",
    "Can you explain the theory?"
]

for prompt in ambiguous_prompts:
    analysis_prompt = f"Analyze the following prompt for ambiguity: '{prompt}'. Explain why it's ambiguous and list possible interpretations."
    print(f"Prompt: {prompt}")
    print(llm.invoke(analysis_prompt).content)
    print("-" * 50)

Prompt: Tell me about the bank.
What a simple yet intriguing prompt!

The prompt "Tell me about the bank" is indeed ambiguous, and here's why:

**Lack of specificity**: The word "bank" can refer to various entities, making it unclear what the speaker is asking about. There are multiple possible interpretations:

1. **Financial institution**: A bank as a financial institution where people deposit and withdraw money, apply for loans, and conduct other financial transactions.
2. **Riverbank**: A bank as a natural feature, a sloping or flat area alongside a body of water, such as a river, lake, or ocean.
3. **Bank (slope)**: A bank as a slope or incline, like a hillside or a mountain slope.
4. **Bank (financial institution) with a specific branch or location**: The speaker might be asking about a particular bank branch, a specific location, or a particular type of bank (e.g., community bank, investment bank, etc.).
5. **Bank (metaphorical)**: The speaker might be using "bank" metaphoricall

#Resolving Ambiguity

In [4]:
import nest_asyncio
nest_asyncio.apply()
import asyncio

In [9]:
async def resolve_ambiguity(prompt, context):
    """
    Resolve ambiguity in a prompt by providing additional context.
    
    Args:
    prompt (str): The original ambiguous prompt
    context (str): Additional context to resolve ambiguity
    
    Returns:
    str: The AI's response to the clarified prompt
    """
    clarified_prompt = f"{context}\n\nBased on this context, {prompt}"
    return await llm.ainvoke(clarified_prompt)

In [11]:
ambiguous_prompt = "Tell me about the bank."
contexts = [
    "You are a financial advisor discussing savings accounts.",
    "You are a geographer describing river formations."
]

for context in contexts:
    print(f"Context: {context}")
    response = asyncio.run(resolve_ambiguity(ambiguous_prompt, context))
    print(f"Clarified response: {response.content}")
    print("-" * 50)

Context: You are a financial advisor discussing savings accounts.
Clarified response: The bank I work with is a reputable and well-established institution with a long history of providing excellent financial services to its customers. With a strong focus on customer satisfaction, they offer a wide range of banking products and services, including savings accounts, checking accounts, loans, credit cards, and investments.

The bank is a FDIC-insured institution, which means that deposits are insured up to $250,000 per account owner, per insured bank. This provides an added layer of security for customers, giving them peace of mind knowing that their hard-earned money is protected.

In terms of savings accounts, the bank offers a variety of options to suit different needs and goals. They have traditional savings accounts with competitive interest rates, as well as high-yield savings accounts that offer higher rates for customers who are willing to keep their money locked in the account fo

#ReWritting Prompt

In [12]:
def compare_prompt_clarity(original_prompt, improved_prompt):
    """
    Compare the responses to an original prompt and an improved, clearer version.
    
    Args:
    original_prompt (str): The original, potentially unclear prompt
    improved_prompt (str): An improved, clearer version of the prompt
    
    Returns:
    tuple: Responses to the original and improved prompts
    """
    original_response = llm.invoke(original_prompt).content
    improved_response = llm.invoke(improved_prompt).content
    return original_response, improved_response


In [13]:
original_prompt = "How do I make it?"
improved_prompt = "Provide a step-by-step guide for making a classic margherita pizza, including ingredients and cooking instructions."

original_response, improved_response = compare_prompt_clarity(original_prompt, improved_prompt)

print("Original Prompt Response:")
print(original_response)
print()
print("\nImproved Prompt Response:")
print(improved_response)

Original Prompt Response:
I'm happy to help! However, I need more information about what you're trying to make. Could you please provide more context or clarify what you're referring to? Are you asking about a specific recipe, a craft project, or something else?


Improved Prompt Response:
The classic margherita pizza! A timeless favorite that's easy to make and always a crowd-pleaser. Here's a step-by-step guide to help you create this iconic Italian dish:

**Ingredients:**

* 1 1/2 cups warm water
* 1 tablespoon active dry yeast
* 3 1/2 cups all-purpose flour
* 1 teaspoon salt
* 2 tablespoons extra-virgin olive oil
* 1/2 cup San Marzano tomatoes (or other fresh, flavorful tomatoes), crushed by hand
* 8 ounces fresh mozzarella cheese, sliced
* Fresh basil leaves, chopped (about 1/4 cup)

**Cooking Instructions:**

**Step 1: Make the Dough**

1. In a large mixing bowl, combine the warm water and yeast. Let it sit for 5-10 minutes, or until the yeast becomes frothy and bubbly.
2. Add th

#Structure prompts for clarity

In [14]:
from langchain_core.prompts import PromptTemplate

structured_prompt = PromptTemplate(
    input_variables=["topic", "aspects", "tone"],
    template="""Provide an analysis of {topic} considering the following aspects:
    1. {{aspects[0]}}
    2. {{aspects[1]}}
    3. {{aspects[2]}}
    
    Present the analysis in a {tone} tone.
    """
)

input_variables = {
    "topic": "the impact of social media on society",
    "aspects": ["communication patterns", "mental health", "information spread"],
    "tone": "balanced and objective"
}

chain = structured_prompt | llm
response = chain.invoke(input_variables).content
print(response)

I'm happy to provide an analysis of the impact of social media on society, considering the following aspects:

**Aspect 1: Mental Health and Well-being**

Social media has had a significant impact on mental health and well-being. On one hand, social media platforms provide a sense of connection and community for many people, which can be particularly important for those who may be isolated or have difficulty forming in-person relationships. Additionally, social media can be a valuable tool for mental health awareness and support, with many people using platforms to share their experiences and connect with others who may be going through similar challenges.

On the other hand, excessive social media use has been linked to increased symptoms of depression, anxiety, and loneliness. Social media platforms can create unrealistic expectations and promote comparison, leading to feelings of inadequacy and low self-esteem. The constant stream of curated and manipulated content can also contribu